# Cuda Programming Project

CEPARCO S11 Group 5 Members:

* Lance Victor Del Rosario
* Audrin Matthew Javier
* Theoni Anne Lim
* Ruth Yee

**Specifications (1D Convolution):**

Input: Two vectors: input vector in and output vector out.

Process: 1D convolution is defined as out[i] = (in[i] + in[i+1] + in[i+2] / 3.0f

Output #1: First and last 20 elements of vector out.

Output #2: Video recording of the assigned CUDA concept (upload on Youtube; Should be “unlisted” and NOT “YouTube Kids”). Link to be placed in Github.


Notes:
* Write the kernel using the specified method in (1) C program; (2) CUDA C program using Colab platform. Place your group number and group members in the first cell.
* CUDA program should use Unified memory, pre-fetching and memadvise.
* Time the kernel portion only with vector size of 228 floating point.
* For each kernel, execute at least 30 times and get the average execution time.
* For the data, initialize each vector with values of your choice. Please document this value.
* Check the correctness of your output. Thus, if the C version is your "sanity check answer key," then the output of the CUDA version must be checked with the C version and output correspondingly (i.e., CUDA
kernel output is correct).

# Visual Profiler

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

--2025-02-20 15:32:20--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.46.228.170, 23.46.228.167, 23.46.228.176
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.46.228.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317705436 (303M) [application/x-deb]
Saving to: ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb.1’

nsight-systems-2023 100%[===================>] 302.99M  53.3MB/s    in 12s     

2025-02-20 15:32:33 (24.7 MB/s) - ‘nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb.1’ saved [317705436/317705436]

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu 

# (1) C Program Version


In [ ]:
%%writefile C_1dconvolution.c

// out[i] = (in[i] + in[i+1] + in[i+2]) / 3.0f

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

// ***C function version
void oned_convolution(size_t n, float* out, float *in)
{
    for (size_t i = 0; i < n - 2; i++)
        out[i] = (in[i] + in[i+1] + in[i+2]) / 3.0f;
}

int main(int argc, char** argv)
{
    const size_t ARRAY_SIZE = 1 << 28;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    // number of times the program is to be executed
    const size_t loope = 30;

    // declare arrays
    float *in, *out;
    in = (float*)malloc(ARRAY_BYTES);
    out = (float*)malloc(ARRAY_BYTES);

    // timer variables
    clock_t start, end;

    // initialize array
    for (size_t i = 0; i < ARRAY_SIZE; i++)
        in[i] = 3.0;

    // fill-in cache
    oned_convolution(ARRAY_SIZE, out, in);

    // time here
    double elapse = 0.0, time_taken;
    for (size_t i = 0; i < loope; i++) {
        start = clock();
        oned_convolution(ARRAY_SIZE, out, in);
        end = clock();
        time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        elapse += time_taken;
    }

    printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n",
           loope, elapse / loope, ARRAY_SIZE);

    // error checking routine
    size_t err_count = 0;
    for (size_t i = 0; i < ARRAY_SIZE-2; i++) {
        if ((in[i] + in[i+1] + in[i+2])/3.0 != out[i])
            err_count++;
    }

    printf("Error count (C program): %lu\n", err_count);


    // Print first and last 20 elements
    printf("First 20 elements: \n");
    for (size_t i = 0; i < 20; i++) {
        printf("%.2f ", out[i]);
    }
    printf("\n");

    printf("Last 20 elements: \n");
    for (size_t i = ARRAY_SIZE - 22; i < ARRAY_SIZE; i++) {
        printf("%.2f ", out[i]);
    }
    printf("\n");


    // Free memory
    free(in);
    free(out);
    return 0;
}


Overwriting C_1dconvolution.c


In [ ]:
%%shell
gcc C_1dconvolution.c -o C_1dconvolution

In [ ]:
%%shell
./C_1dconvolution

Function (in C) average time for 30 loops is 1002.400533 milliseconds to execute an array size 268435456 
Error count (C program): 0
First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 


# (2) CUDA Program Version

In [ ]:
%%writefile CUDA_1dconvolution.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// CUDA 1D convolution kernel
__global__
void oned_convolution(size_t n, float* out, float* in) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = index; i < n - 2; i += stride) {
        out[i] = (in[i] + in[i+1] + in[i+2]) / 3.0f;
    }
}

int main() {
    const size_t ARRAY_SIZE = 1 << 28;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loope = 30;

    float *in, *out;
    cudaMallocManaged(&in, ARRAY_BYTES);
    cudaMallocManaged(&out, ARRAY_BYTES);

    // Get GPU ID
    int device = -1;
    cudaGetDevice(&device);

    // Memory advice
    cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

    // Prefetch data to CPU
    cudaMemPrefetchAsync(in, ARRAY_BYTES, cudaCpuDeviceId, NULL);
    // Prefetch data to GPU
    cudaMemPrefetchAsync(out, ARRAY_BYTES, device, NULL);

    // Initialize input array
    for (size_t i = 0; i < ARRAY_SIZE; i++)
        in[i] = 3.0;

    // Prefetch data from CPU to GPU
    cudaMemPrefetchAsync(in, ARRAY_BYTES, device, NULL);

    // Set up CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE - 2 + numThreads - 1) / numThreads;

    printf("*** Function: \n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu\n", numBlocks, numThreads);

    for (size_t i = 0; i < loope; i++) {
        oned_convolution<<<numBlocks, numThreads>>>(ARRAY_SIZE, out, in);
    }

    // Barrier
    cudaDeviceSynchronize();

    // Prefetch data from GPU to CPU
    cudaMemPrefetchAsync(out, ARRAY_BYTES, cudaCpuDeviceId, NULL);
    // cudaMemPrefetchAsync(in, ARRAY_BYTES, cudaCpuDeviceId, NULL);

    // Print first and last 20 elements
    printf("First 20 elements: \n");
    for (size_t i = 0; i < 20; i++) {
        printf("%.2f ", out[i]);
    }
    printf("\n");

    printf("Last 20 elements: \n");
    for (size_t i = ARRAY_SIZE - 22; i < ARRAY_SIZE; i++) {
        printf("%.2f ", out[i]);
    }
    printf("\n");

    // Error check
    size_t err_count = 0;
    for (size_t i = 0; i < ARRAY_SIZE-2; i++) {
        if (abs((in[i] + in[i+1] + in[i+2]) / 3.0f - out[i]) > 0.1) {
            err_count++;
        }
    }
    printf("Error count (CUDA program): %zu\n", err_count);

    // Free memory
    cudaFree(in);
    cudaFree(out);


    return 0;
}

Writing CUDA_1dconvolution.cu


In [ ]:
%%shell
nvcc -arch=sm_75 CUDA_1dconvolution.cu -o CUDA_1dconvolution

In [ ]:
%%shell
nvprof ./CUDA_1dconvolution

==5748== NVPROF is profiling process 5748, command: ./CUDA_1dconvolution
*** Function: 
numElements = 268435456
numBlocks = 262144, numThreads = 1024
First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 
Error count (CUDA program): 0
==5748== Profiling application: ./CUDA_1dconvolution
==5748== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  335.86ms        30  11.195ms  8.7884ms  14.792ms  oned_convolution(unsigned long, float*, float*)
      API calls:   38.99%  425.87ms         4  106.47ms  4.1342ms  239.11ms  cudaMemPrefetchAsync
                   30.74%  335.78ms         1  335.78ms  335.78ms  335.78ms  cudaDeviceSynchronize
                   18.52%  202.25ms         2  101.13ms  75.403us  202.18ms  cudaMall

In [ ]:
%%shell
nsys profile ./CUDA_1dconvolution

*** Function: 
numElements = 268435456
numBlocks = 262144, numThreads = 1024
First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 
Error count (CUDA program): 0
Generating '/tmp/nsys-report-8041.qdstrm'
[1/1] [========================100%] report3.nsys-rep
Generated:
    /content/report3.nsys-rep


# (3) CUDA with Streams and Prefetching, Mem Advise, Unified Memory

In [ ]:
%%writefile CUDAStream_1dconvolution.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define NUM_STREAMS 4  // Number of CUDA streams
#define OVERLAP 2      // Extra elements to ensure correct convolution at boundaries

// CUDA 1D convolution kernel
__global__
void oned_convolution(size_t n, float* out, float* in) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = index; i < n - 2; i += stride) {
        out[i] = (in[i] + in[i+1] + in[i+2]) / 3.0f;
    }
}

int main() {
    const size_t ARRAY_SIZE = 1 << 28;  // 16M elements
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loope = 30;

    float *in, *out;
    cudaMallocManaged(&in, ARRAY_BYTES);
    cudaMallocManaged(&out, ARRAY_BYTES);

    // Get GPU ID
    int device = -1;
    cudaGetDevice(&device);

    // Mem advise
    cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);

    //prefetch data to CPU page memory
    cudaMemPrefetchAsync(in, ARRAY_BYTES, cudaCpuDeviceId, NULL);
    //prefetch data to GPU page memory
    cudaMemPrefetchAsync(out, ARRAY_BYTES, device, NULL);

    // Initialize input array
    for (size_t i = 0; i < ARRAY_SIZE; i++)
        in[i] = 3.0f;

    // Prefetch input data to GPU
    cudaMemPrefetchAsync(in, ARRAY_BYTES, device, NULL);

    // Set up CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE - 2 + numThreads - 1) / numThreads;
    size_t segmentSize = ARRAY_SIZE / NUM_STREAMS;

    printf("*** Function: \n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu\n", numBlocks, numThreads);
    printf("Using %d CUDA Streams\n", NUM_STREAMS);

    // Create CUDA streams
    cudaStream_t streams[NUM_STREAMS];
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamCreate(&streams[i]);
    }

    for (size_t i = 0; i < loope; i++) {
      for (int s = 0; s < NUM_STREAMS; s++) {
          size_t startIdx = s * segmentSize;
          size_t endIdx = startIdx + segmentSize;

          // Ensure overlap handling
          if (s > 0) startIdx -= OVERLAP;
          if (s < NUM_STREAMS - 1) endIdx += OVERLAP;

          size_t segmentElements = endIdx - startIdx;
          size_t segmentBytes = segmentElements * sizeof(float);

          // Launch kernel in different streams
          oned_convolution<<<(segmentElements - 2 + numThreads - 1) / numThreads, numThreads, 0, streams[s]>>>(
              segmentElements, out + startIdx, in + startIdx);

      }
    }

    // Synchronize all streams
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamSynchronize(streams[i]);
    }

    // Prefetch data from GPU to CPU
    cudaMemPrefetchAsync(in, ARRAY_BYTES, cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(out, ARRAY_BYTES, cudaCpuDeviceId, NULL);

    // Print first and last 20 elements
    printf("First 20 elements: \n");
    for (size_t i = 0; i < 20; i++) {
        printf("%.2f ", out[i]);
    }
    printf("\n");

    printf("Last 20 elements: \n");
    for (size_t i = ARRAY_SIZE - 22; i < ARRAY_SIZE; i++) {
        printf("%.2f ", out[i]);
    }
    printf("\n");

    // Error check
    size_t err_count = 0;
    for (size_t i = OVERLAP; i < ARRAY_SIZE - OVERLAP - 2; i++) {  // Ignore overlap edges
        float expected = (in[i] + in[i+1] + in[i+2]) / 3.0f;
        if (abs(expected - out[i]) > 0.1) {
            err_count++;
        }
    }
    printf("Error count (CUDA program): %zu\n", err_count);

    // Free memory
    for(int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamDestroy(streams[i]);
    }
    cudaFree(in);
    cudaFree(out);

    return 0;
}


Overwriting CUDAStream_1dconvolution.cu


In [ ]:
%%shell
nvcc -arch=sm_75 CUDAStream_1dconvolution.cu -o CUDAStream_1dconvolution

CUDAStream_1dconvolution.cu(74): warning #177-D: variable "segmentBytes" was declared but never referenced
            size_t segmentBytes = segmentElements * sizeof(float);
                   ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [ ]:
%%shell
nvprof ./CUDAStream_1dconvolution

==7331== NVPROF is profiling process 7331, command: ./CUDAStream_1dconvolution
*** Function: 
numElements = 268435456
numBlocks = 262144, numThreads = 1024
Using 4 CUDA Streams
First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 
Error count (CUDA program): 0
==7331== Profiling application: ./CUDAStream_1dconvolution
==7331== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  317.57ms       120  2.6464ms  2.2024ms  3.7114ms  oned_convolution(unsigned long, float*, float*)
      API calls:   41.19%  429.11ms         5  85.822ms  733.46us  239.08ms  cudaMemPrefetchAsync
                   30.41%  316.83ms         4  79.208ms  2.4059ms  309.57ms  cudaStreamSynchronize
                   21.87%  227.87ms         2  113.9

Run this to generate a downloadable file to view in the NSight Profiler

In [ ]:
%%shell
nsys profile ./CUDAStream_1dconvolution

*** Function: 
numElements = 16777216
numBlocks = 16384, numThreads = 1024
Using 4 CUDA Streams
First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 
Error count (CUDA program): 0
Generating '/tmp/nsys-report-afd3.qdstrm'
[1/1] [========================100%] report4.nsys-rep
Generated:
    /content/report4.nsys-rep


# (4) CUDA with Streams and Memcpy (no prefetching, mem advise, or unified memory)

In [ ]:
%%writefile CUDAstreamonly_1dconvolution.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define NUM_STREAMS 4
#define OVERLAP 2
__global__
void oned_convolution(size_t n, float* out, float* in) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < n - 2)
        out[i] = (in[i] + in[i+1] + in[i+2]) / 3.0f;
}

int main(){
    const size_t ARRAY_SIZE = 1 << 28;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loope = 30;

    //initialize and allocate
    float *host_in, *host_out;
    float *dev_in, *dev_out;
    //host_in = (float*)malloc(ARRAY_BYTES);
    //host_out = (float*)malloc(ARRAY_BYTES);

    cudaMallocHost((void**)&host_in, ARRAY_BYTES);
    cudaMallocHost((void**)&host_out, ARRAY_BYTES);

    //initialize values
    for(size_t i = 0; i < ARRAY_SIZE; i++){
      host_in[i] = 3.0;
    }

    cudaMalloc((void**)&dev_in, ARRAY_BYTES);
    cudaMalloc((void**)&dev_out, ARRAY_BYTES);

    //stream creation and running
    cudaStream_t streams[NUM_STREAMS];
    for(int i = 0; i < NUM_STREAMS; i++){
      cudaStreamCreate(&streams[i]);
    }


    //event creation
    cudaEvent_t event_start[NUM_STREAMS], event_end[NUM_STREAMS];
    for(int i = 0; i < NUM_STREAMS; i++){
      cudaEventCreate(&event_start[i]);
      cudaEventCreate(&event_end[i]);
    }

    size_t numThreads = 1024;
    //size_t numBlocks = (ARRAY_SIZE - 2 + numThreads - 1) / numThreads;
    size_t segmentSize = ARRAY_SIZE / NUM_STREAMS;
    //size_t segmentBytes = segmentSize * sizeof(float);

    //Host to Device
    for(int i = 0; i < NUM_STREAMS; i++) {
      size_t startIdx = i * segmentSize;
      size_t endIdx = startIdx + segmentSize;

      if (i > 0) startIdx -= OVERLAP;
      if (i < NUM_STREAMS - 1) endIdx += OVERLAP;

      size_t segmentElements = endIdx - startIdx;
      size_t segmentBytes = segmentElements * sizeof(float);

      cudaMemcpyAsync(dev_in + startIdx, host_in + startIdx, segmentBytes, cudaMemcpyHostToDevice, streams[i]);

      cudaEventRecord(event_start[i], streams[i]);
    }

    //cudaDeviceSynchronize();

    //actual kernel run on multiple streams
    //for(size_t i = 0; i < loope; i++){
      for(int s = 0; s < NUM_STREAMS; s++){
        size_t startIdx = s * segmentSize;
          size_t endIdx = startIdx + segmentSize;

          // Ensure overlap handling
          if (s > 0) startIdx -= OVERLAP;
          if (s < NUM_STREAMS - 1) endIdx += OVERLAP;

          size_t segmentElements = endIdx - startIdx;
          //size_t segmentBytes = segmentElements * sizeof(float);

          cudaStreamWaitEvent(streams[s], event_start[s], 0);

          // Launch kernel in different streams
            oned_convolution<<<(segmentElements - 2 + numThreads - 1) / numThreads, numThreads, 0, streams[s]>>>(segmentElements, dev_out + startIdx, dev_in + startIdx);

            cudaEventRecord(event_end[s], streams[s]);
      }
    //}

    // Device to Host

    for (int i = 0; i < NUM_STREAMS; i++) {
        size_t startIdx = i * segmentSize;
        size_t endIdx = startIdx + segmentSize;

        if (i > 0) startIdx -= OVERLAP;
        if (i < NUM_STREAMS - 1) endIdx += OVERLAP;

        size_t segmentElements = endIdx - startIdx;
        size_t segmentBytes = segmentElements * sizeof(float);

        cudaStreamWaitEvent(streams[i], event_end[i], 0);
        cudaMemcpyAsync(host_out + startIdx, dev_out + startIdx, segmentBytes, cudaMemcpyDeviceToHost, streams[i]);
    }

    //synching
    cudaDeviceSynchronize();



    //print first and last 20 elements
    printf("First 20 elements: \n");
    for (size_t i = 0; i < 20; i++) {
        printf("%.2f ", host_out[i]);
    }
    printf("\n");

    printf("Last 20 elements: \n");
    for (size_t i = ARRAY_SIZE - 22; i < ARRAY_SIZE; i++) {
        printf("%.2f ", host_out[i]);
    }
    printf("\n");

    // Error check
    size_t err_count = 0;
    for (size_t i = 0; i < ARRAY_SIZE-2; i++) {
        if (abs((host_in[i] + host_in[i+1] + host_in[i+2]) / 3.0f - host_out[i]) > 0.1) {
            err_count++;
        }
    }
    printf("Error count (CUDA program): %zu\n", err_count);

    //cleanup
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamDestroy(streams[i]);
        cudaEventDestroy(event_start[i]);
        cudaEventDestroy(event_end[i]);
    }
    cudaFree(dev_in);
    cudaFree(dev_out);
    cudaFreeHost(host_in);
    cudaFreeHost(host_out);
    return 0;
}

Overwriting CUDAstreamonly_1dconvolution.cu


In [ ]:
%%shell
nvcc -arch=sm_75 CUDAstreamonly_1dconvolution.cu -o CUDAstreamonly_1dconvolution

CUDAstreamonly_1dconvolution.cu(19): warning #177-D: variable "loope" was declared but never referenced
      const size_t loope = 30;
                   ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [ ]:
%%shell
nvprof ./CUDAstreamonly_1dconvolution

==6957== NVPROF is profiling process 6957, command: ./CUDAstreamonly_1dconvolution
First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 
Error count (CUDA program): 0
==6957== Profiling application: ./CUDAstreamonly_1dconvolution
==6957== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   49.57%  92.833ms         4  23.208ms  21.932ms  26.138ms  [CUDA memcpy HtoD]
                   42.73%  80.027ms         4  20.007ms  19.922ms  20.172ms  [CUDA memcpy DtoH]
                    7.70%  14.429ms         4  3.6072ms  3.6012ms  3.6214ms  oned_convolution(unsigned long, float*, float*)
      API calls:   60.96%  1.21269s         2  606.35ms  499.03ms  713.66ms  cudaMallocHost
                   29.91%  594.95ms         2  297.48ms

In [ ]:
%%shell
nsys profile ./CUDAstreamonly_1dconvolution

First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 
Error count (CUDA program): 0
Generating '/tmp/nsys-report-aa19.qdstrm'
[1/1] [========================100%] report5.nsys-rep
Generated:
    /content/report5.nsys-rep


# (5) CUDA with Streams and Memcpy for each loope (no prefetching, mem advise, or unified memory)



In [ ]:
%%writefile test.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define NUM_STREAMS 4
#define OVERLAP 2

__global__
void oned_convolution(size_t n, float* out, float* in) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < n - 2)
        out[i] = (in[i] + in[i+1] + in[i+2]) / 3.0f;
}

int main() {
    const size_t ARRAY_SIZE = 1 << 28; // 26M elements
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loope = 30;

    // Initialize and allocate
    float *host_in, *host_out;
    float *dev_in, *dev_out;

    // Use pinned memory for better transfer performance
    cudaMallocHost((void**)&host_in, ARRAY_BYTES);
    cudaMallocHost((void**)&host_out, ARRAY_BYTES);

    // Initialize input values
    for (size_t i = 0; i < ARRAY_SIZE; i++) {
        host_in[i] = 3.0;
    }

    // Allocate device memory
    cudaMalloc((void**)&dev_in, ARRAY_BYTES);
    cudaMalloc((void**)&dev_out, ARRAY_BYTES);

    // Create streams and events
    cudaStream_t streams[NUM_STREAMS];
    cudaEvent_t event_start[NUM_STREAMS], event_end[NUM_STREAMS];

    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamCreate(&streams[i]);
        cudaEventCreate(&event_start[i]);
        cudaEventCreate(&event_end[i]);
    }

    size_t numThreads = 1024;
    size_t segmentSize = ARRAY_SIZE / NUM_STREAMS;

    // Main loop for overlapping memory transfers and kernel execution
    for (size_t i = 0; i < loope + 2; i++) {
        for (int s = 0; s < NUM_STREAMS; s++) {
            size_t startIdx = s * segmentSize;
            size_t endIdx = startIdx + segmentSize;

            // Adjust for overlap
            if (s > 0) startIdx -= OVERLAP;
            if (s < NUM_STREAMS - 1) endIdx += OVERLAP;

            size_t segmentElements = endIdx - startIdx;
            size_t segmentBytes = segmentElements * sizeof(float);

            // Copy data to device
            cudaMemcpyAsync(dev_in + startIdx, host_in + startIdx, segmentBytes, cudaMemcpyHostToDevice, streams[s]);

            // Record event after memory transfer
            cudaEventRecord(event_start[s], streams[s]);

            // Wait for memory transfer to complete before launching kernel
            cudaStreamWaitEvent(streams[s], event_start[s], 0);

            // Launch kernel
            oned_convolution<<<(segmentElements - 2 + numThreads - 1) / numThreads, numThreads, 0, streams[s]>>>(segmentElements, dev_out + startIdx, dev_in + startIdx);

            // Record event after kernel execution
            cudaEventRecord(event_end[s], streams[s]);

            // Wait for kernel to complete before copying data back
            cudaStreamWaitEvent(streams[s], event_end[s], 0);

            // Copy data back to host
            cudaMemcpyAsync(host_out + startIdx, dev_out + startIdx, segmentBytes, cudaMemcpyDeviceToHost, streams[s]);
        }
    }

    // Synchronize all streams at the end
    cudaDeviceSynchronize();

    // Print first and last 20 elements
    printf("First 20 elements: \n");
    for (size_t i = 0; i < 20; i++) {
        printf("%.2f ", host_out[i]);
    }
    printf("\n");

    printf("Last 20 elements: \n");
    for (size_t i = ARRAY_SIZE - 22; i < ARRAY_SIZE; i++) {
        printf("%.2f ", host_out[i]);
    }
    printf("\n");

    // Error check
    size_t err_count = 0;
    for (size_t i = 0; i < ARRAY_SIZE - 2; i++) {
        if (abs((host_in[i] + host_in[i+1] + host_in[i+2]) / 3.0f - host_out[i]) > 0.1) {
            err_count++;
        }
    }
    printf("Error count (CUDA program): %zu\n", err_count);

    // Cleanup
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamDestroy(streams[i]);
        cudaEventDestroy(event_start[i]);
        cudaEventDestroy(event_end[i]);
    }
    cudaFree(dev_in);
    cudaFree(dev_out);
    cudaFreeHost(host_in);
    cudaFreeHost(host_out);

    return 0;
}

Overwriting test.cu


In [ ]:
%%shell
nvcc -arch=sm_75 test.cu -o test

In [ ]:
%%shell
nvprof ./test

==6121== NVPROF is profiling process 6121, command: ./test
First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 
Error count (CUDA program): 0
==6121== Profiling application: ./test
==6121== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   47.95%  3.58320s       128  27.994ms  23.727ms  30.685ms  [CUDA memcpy HtoD]
                   47.62%  3.55850s       128  27.801ms  19.932ms  30.851ms  [CUDA memcpy DtoH]
                    4.42%  330.48ms       128  2.5819ms  2.3671ms  3.6282ms  oned_convolution(unsigned long, float*, float*)
      API calls:   66.57%  3.57907s         1  3.57907s  3.57907s  3.57907s  cudaDeviceSynchronize
                   21.86%  1.17515s         2  587.58ms  489.39ms  685.77ms  cudaMallocHost
    

In [ ]:
%%shell
nsys profile ./test

First 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 
Last 20 elements: 
3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 3.00 0.00 0.00 
Error count (CUDA program): 0
Generating '/tmp/nsys-report-0150.qdstrm'
[1/1] [========================100%] report6.nsys-rep
Generated:
    /content/report6.nsys-rep
